##Read data from bronze table


In [0]:
from pyspark.sql import functions as F

df_bronze = spark.table("lakehouse.bronze.sales_bronze")
display(df_bronze)


##Cleaning and schema fixes


In [0]:
df_silver = (
    df_bronze
    # safely convert string date to proper date (format like 12/1/2024)
    .withColumn(
        "order_date",
        F.try_to_date("order_date", "M/d/yyyy")
    )
    .withColumn("quantity", F.col("quantity").cast("int"))
    .withColumn("price", F.col("price").cast("double"))
    .withColumn("total_amount", F.col("quantity") * F.col("price"))
)

display(df_silver)
df_silver.printSchema()

##Save cleaned data to silver table 

In [0]:
df_silver.write.mode("overwrite").saveAsTable("lakehouse.silver.sales_silver")

In [0]:
%sql
select * from lakehouse.silver.sales_silver;